In [1]:
import sys 
sys.path.append("../")

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torch.nn import BCELoss

from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

from pathlib import *
import os
import tqdm
import argparse
import json

from models.tcn import MS_TCN, SS_TCN
from dataset.VolvoDataset import VolvoDatasetPart1, VolvoDatasetPart2
from utils.ContinuityCrossEntropyLoss import ContinuityCrossEntropyLoss
from utils.StatsComputer import StatsComputer

In [17]:
tensor = torch.tensor([0.1, 0.2, 0, 0.4, 0.5])
tensor_1 = torch.tensor([0.1, 0.2, 0.3, 0.4, 0.5])

def tensor_to_vector(tensor):
    tensor = tensor * 10
    tensor = torch.round(tensor)
    tensor

    # for each number, create a tensor with 10 elements, with (1 - number) being 0 and number being 1
    list = []
    for n in tensor:
        list.append(torch.cat([torch.zeros(int(10 - n)), torch.ones(int(n))]))

    return torch.stack(list)

tensor = tensor_to_vector(tensor)
tensor_1 = tensor_to_vector(tensor_1)

# count total accuracy

correct = torch.eq(tensor, tensor_1).sum().item()
total = tensor.numel()
correct / total
    

0.94

In [15]:
tensor, tensor_1

(tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0., 1., 1.],
         [0., 0., 0., 0., 0., 0., 0., 1., 1., 1.],
         [0., 0., 0., 0., 0., 0., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 1., 1., 1., 1., 1.]]),
 tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0., 0., 1., 1., 1.],
         [0., 0., 0., 0., 0., 0., 0., 1., 1., 1.],
         [0., 0., 0., 0., 0., 0., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 1., 1., 1., 1., 1.]]))

In [2]:
args = argparse.Namespace(
    seed=13_04_2000,
    test_only=False,
    load_model="",
    batch_size=256,
    num_epochs=30,
    learning_rate=0.0005,
    lr_scheduler_gamma=0.8,
    lr_scheduler_step=1,
    patience_epochs=7,
    disable_cuda=False,
    data_path=r"/data1/malto/volvo_ecml_2024",
    train_csv=r"train_gen1.csv",
    test_csv=r"public_X_test.csv",
    variants_csv=r"variants.csv",
    tcnn_weights=r"./"
)

In [3]:
args = args

np.random.seed(args.seed)
torch.manual_seed(args.seed)

### Get important paths
dataset_path = args.data_path 
variants_path = os.path.join(dataset_path, args.variants_csv)
train_data_path = os.path.join(dataset_path, args.train_csv)
test_data_path = os.path.join(dataset_path, args.test_csv)
weights_path = args.tcnn_weights
os.makedirs(weights_path, exist_ok=True)

### Get dataset and model type
train_data_path = os.path.join(args.data_path, "train_gen1.csv")
test_data_path = os.path.join(args.data_path, "public_X_test.csv")
variants_path = os.path.join(args.data_path, "variants.csv")

In [4]:
train_dataset = VolvoDatasetPart1(data_path=train_data_path, variants_path=variants_path)
processor = train_dataset.get_processor()
label_encoder = processor.risk_encoder

train_dataset, validation_dataset = train_dataset.split_train_validation()

test_dataset = VolvoDatasetPart1(data_path=test_data_path, variants_path=variants_path, test=True)
test_dataset.set_processor(processor) 

100%|██████████| 3359/3359 [00:00<00:00, 104593.70it/s]


In [5]:
x, x_static, y = train_dataset[0]

In [6]:
y

tensor([1., 0.])

In [7]:
train_dataset.volvo_df

,Timesteps,ChassisId_encoded,gen,risk_level,af1__0,af1__1,af1__2,af1__3,af1__4,af1__6,...,f__232,f__233,f__234,f__235,f__237,f__238,f__239,f__240,f__241,f__242
0,0.0,4953.0,gen1,High,0.000000,0.00000,0.000000,0.000000e+00,0.000000,0.000000,...,-5.694897,-5.827349,-1.449851,-0.350315,1.394857,0.192880,-0.495562,-0.928794,-0.214402,-0.090670
1,0.0,4955.0,gen1,Low,0.000322,0.00002,0.057749,8.627059e-08,0.029966,0.019837,...,0.121147,0.230757,0.280036,-0.206586,-0.088550,-0.836415,-0.484087,0.320285,2.311401,-0.026043
2,1.0,4955.0,gen1,Low,0.000322,0.00002,0.057749,8.627059e-08,0.029966,0.019837,...,0.121147,0.230757,0.280036,-0.206586,-0.091570,-0.841418,-0.472325,0.322941,2.284491,-0.026732
3,2.0,4955.0,gen1,Low,0.000322,0.00002,0.057749,8.627059e-08,0.029966,0.019837,...,0.121147,0.230757,0.280036,-0.206586,-0.029233,-0.817824,-0.509273,0.306841,2.266275,-0.027429
4,3.0,4955.0,gen1,Low,0.000322,0.00002,0.057749,8.627059e-08,0.029966,0.019837,...,0.121147,0.230757,0.280036,-0.206586,-0.002841,-0.808023,-0.513796,0.309057,2.237995,-0.028150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157432,12.0,102478.0,gen1,High,0.000000,0.00000,0.000000,0.000000e+00,0.000000,0.000000,...,-5.694897,-5.827349,-1.449851,-0.350315,-0.513768,-2.615375,1.035627,1.758682,0.122545,-0.088263
157433,13.0,102478.0,gen1,High,0.000000,0.00000,0.000000,0.000000e+00,0.000000,0.000000,...,-5.694897,-5.827349,-1.449851,-0.350315,-0.511595,-2.614036,1.043629,1.750753,0.114332,-0.088321
157434,14.0,102478.0,gen1,High,0.000000,0.00000,0.000000,0.000000e+00,0.000000,0.000000,...,-5.694897,-5.827349,-1.449851,-0.350315,-0.511128,-2.605104,1.058136,1.737135,0.102352,-0.088407
157435,15.0,102478.0,gen1,High,0.000000,0.00000,0.000000,0.000000e+00,0.000000,0.000000,...,-5.694897,-5.827349,-1.449851,-0.350315,-0.508548,-2.607412,1.061236,1.741602,0.098320,-0.088435


In [8]:
"""train_dataset = VolvoDatasetPart2(data_path=train_data_path, variants_path=variants_path)
processor = train_dataset.get_processor()
label_encoder = processor.risk_encoder

train_dataset, validation_dataset = train_dataset.split_train_validation()

test_dataset = VolvoDatasetPart2(data_path=test_data_path, variants_path=variants_path, test=True)
test_dataset.set_processor(processor) """

'train_dataset = VolvoDatasetPart2(data_path=train_data_path, variants_path=variants_path)\nprocessor = train_dataset.get_processor()\nlabel_encoder = processor.risk_encoder\n\ntrain_dataset, validation_dataset = train_dataset.split_train_validation()\n\ntest_dataset = VolvoDatasetPart2(data_path=test_data_path, variants_path=variants_path, test=True)\ntest_dataset.set_processor(processor) '

In [9]:
n_features = train_dataset.get_n_features()
num_classes = train_dataset.get_n_classes()

#check if preprocess is giving some problems
assert train_dataset.get_n_features() == test_dataset.get_n_features()

model = SS_TCN(     num_input_channels=n_features, 
                    num_classes=num_classes, 
                    is_phase_1=True)

### Get device
device = torch.device(
            "cuda" if (torch.cuda.is_available() and not args.disable_cuda) else "cpu"
        )
model.to(device)
print(f"Working on {device}")

# Load weights if necessary
if args.load_model != "":
    if not(args.load_model.endswith(".pth") or args.load_model.endswith(".pt")):
        raise Exception("Weights file should end with .pt or .pth")
    model_path = os.join.path(weights_path, args.load_model)
    print(f"Loading Model from {model_path}")
    model.load_state_dict(
        torch.load( model_path )
    )

# Create DataLoader instances for train, validation, and test sets
train_loader = DataLoader(train_dataset, 
                                batch_size=args.batch_size, 
                                shuffle=True,
                                num_workers=12) #pin_memory=True #consigliano
val_loader = DataLoader(validation_dataset, 
                                batch_size=args.batch_size, 
                                shuffle=True,
                                num_workers=12)
test_loader =  DataLoader(test_dataset, 
                                batch_size=args.batch_size)

# Define criterion
print('Computing class weights...', end='')
weights = train_dataset.get_weights()
criterion = BCELoss(weight=torch.Tensor(weights).to(device))
optimizer = optim.Adam(model.parameters(), lr=args.learning_rate)
print('done')
print('Class weights = ', weights)
# criterion = ContinuityCrossEntropyLoss(weights=torch.Tensor([1,1,1]).to(device))

Working on cuda
Computing class weights...done
Class weights =  [0.8785872 0.1214128]


In [10]:
print("=== Start training ===")
print(f"Batch size: {args.batch_size}")
# Define loss function and optimizer
optimizer = optim.Adam(model.parameters(), lr=args.learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, gamma=args.lr_scheduler_gamma, step_size=args.lr_scheduler_step)
softmax = torch.nn.functional.softmax
waiting_epochs = 0
best_val_loss = float('inf')
best_val_f1 = 0
num_resets = 0
for epoch in range(args.num_epochs):
    ### Run epoch
    print( "="*25, f"EPOCH {epoch}", "="*25)
    print("Learning rate: ", optimizer.param_groups[0]['lr'])
    print("=== TRAIN ===")
    model.train()     
    pbar = tqdm.tqdm(train_loader)
    running_loss = 0
    i = 0
    for timeseries, variants, labels in pbar:
        pbar.set_description(f"Running loss: {running_loss/(i+1e-5) :.4}")
        timeseries, variants, labels = timeseries.to(device), variants.to(device), labels.to(device)
        outputs = model(timeseries, variants)
        outputs = softmax(outputs, dim=-1)
        optimizer.zero_grad()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        i += 1

    epoch_loss = running_loss / (i+1e-5)
    scheduler.step()
    print(f"Epoch [{epoch+1}/{args.num_epochs}], Train Loss: {epoch_loss:.4f}")

    print("=== VAL ===")
    model.eval()
    with torch.no_grad():
        pbar = tqdm.tqdm(val_loader)
        running_loss = 0
        running_acc = 0
        i = 0
        stats = StatsComputer()
        for timeseries, variants, labels in pbar:
            pbar.set_description(f"Running loss: {running_loss/(i+1e-5) :.4}")
            
            timeseries, variants, labels = timeseries.to(device), variants.to(device), labels.to(device)

            outputs = model(timeseries, variants)
            outputs = softmax(outputs, dim=-1)
            loss = criterion(outputs, labels)
            
            # outputs = outputs[-1]
            acc = torch.sum(torch.argmax(labels, dim = 1) == torch.argmax(outputs, dim = 1))
            
            running_acc += acc 

            stats.append(outputs=torch.argmax(outputs, dim=-1).cpu().tolist(), 
                        labels=torch.argmax(labels, dim=-1).cpu().tolist())

            i += 1

        validation_loss = running_loss / i
        validation_accuracy = running_acc / i
        
        validation_f1 = stats.macro_avg_f1()
        print(stats)
    
    print(f"Validation Accuracy: {validation_accuracy:.4f}")
    print(f"Validation Loss: {validation_loss:.4f} vs Best {best_val_loss:.4f}")
    print(f"Validation F1: {validation_f1} vs Best {best_val_f1}")


=== Start training ===
Batch size: 256
========================= EPOCH 0 =========================
Learning rate:  0.0005
=== TRAIN ===


Running loss: 0.0:   0%|          | 0/74 [00:03<?, ?it/s]/home/grosso/.conda/envs/jack_predicting-failure-risk-levels/lib/python3.12/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
Running loss: 0.3269:  97%|█████████▋| 72/74 [00:26<00:00,  4.61it/s]/home/grosso/.conda/envs/jack_predicting-failure-risk-levels/lib/python3.12/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
Running l

Epoch [1/30], Train Loss: 0.3261
=== VAL ===


Running loss: 0.0: 100%|██████████| 19/19 [00:07<00:00,  2.53it/s]


[0.98990363 0.84397163]
[0.98990363 0.84397163]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4344
           1       0.89      0.80      0.84       296

    accuracy                           0.98      4640
   macro avg       0.94      0.90      0.92      4640
weighted avg       0.98      0.98      0.98      4640

Average discontinuity = 25.47
More than max = tensor([0.0000, 5.5789, 0.0000])
F1 score = 0.92

Validation Accuracy: 239.5789
Validation Loss: 0.0000 vs Best inf
Validation F1: 0.92 vs Best 0
========================= EPOCH 1 =========================
Learning rate:  0.0004
=== TRAIN ===


Running loss: 0.2571:  49%|████▊     | 36/74 [00:17<00:18,  2.01it/s]


KeyboardInterrupt: 

In [12]:
from sklearn.metrics import f1_score, precision_recall_fscore_support
def flatten(xss):
    return [x for xs in xss for x in xs]
    
precision, recall, f1, true_sum = precision_recall_fscore_support(flatten(stats.all_labels), flatten(stats.all_outputs)),

ValueError: not enough values to unpack (expected 4, got 1)